In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
#!pip install langchain langchain-community faiss-cpu PyPDF2 sentence-transformers

In [6]:
#!pip install ollama

In [ ]:
import os
import glob
import re
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_community.llms import Ollama
from typing import List
from langchain.prompts import PromptTemplate

pdf_directory = "./pdf"
txt_directory = "./txt"

documents = []

pdf_files = glob.glob(os.path.join(pdf_directory, "*.pdf"))
print(f"{len(pdf_files)}개의 PDF 파일을 발견했습니다.")

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

txt_files = glob.glob(os.path.join(txt_directory, "*.txt"))
print(f"{len(txt_files)}개의 TXT 파일을 발견했습니다.")

for txt_file in txt_files:
    loader = TextLoader(txt_file, encoding='utf-8')  # 한글 인코딩 지원
    documents.extend(loader.load())
print(f"총 {len(documents)}개의 문서 로드 완료")

def clean_text(text: str) -> str:
    text = text.replace('\xa0', ' ')
    text = re.sub(r'http\S+', '', text)             # url 제거
    text = re.sub(r'\s+', ' ', text)                # 긴여백 제거
    text = re.sub(r'[^\w\s.,!?()\[\]{}가-힣A-Za-z0-9]', '', text)  #특수문자 제거
    return text.strip()


for i, doc in enumerate(documents):
    cleaned_content = clean_text(doc.page_content)
    documents[i].page_content = cleaned_content
    # print(doc.metadata['source'],'|',documents[i].page_content)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)
print(f"총 {len(chunks)}개의 청크로 분할 완료")


# 확인 결과
# 1. 이미지로만 이뤄진 pdf는 제목만 뽑히고 내용은 없어서 삭제(홍콩을 여행할 때 알아둘 점.pdf, 기내반입절차 포스터(영문포함)_배포용.pdf)    
# 2. 띄어쓰기 엉망인 친구들 삭제 ([미국생활 꿀팁] 한국과 다른 미국 생활 미리 알면 삶이 쉬워진다_.pdf, 운송제한 물품│아시아나항공.pdf)
# 3. 띄어씌가 '제목'만 이상한 놈 (싱가포르 여행 시 주의사항, 반드시 알아두자!.pdf)

31개의 PDF 파일을 발견했습니다.
18개의 TXT 파일을 발견했습니다.
총 227개의 문서 로드 완료
총 328개의 청크로 분할 완료


In [31]:
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},     # GPU 있으면 'cuda'로 변경
    encode_kwargs={'normalize_embeddings': True}
)

In [32]:
# FAISS 벡터 저장소 생성
vector_store = FAISS.from_documents(chunks, embeddings)

# 벡터 저장소 저장 (나중에 재사용할 수 있도록)
vector_store.save_local("faiss_index") 
print("벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.")

# Ollama 모델 설정 (gemma3:1b 사용)
# llm = Ollama(model="gemma3:1b")

벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.


In [ ]:
# 백터 예시
query = '우리나라와 다른 나라의 문화차이'
retrieval = vector_store.similarity_search_with_score(query)
print(retrieval)

[(Document(id='ac4d031b-aaa9-4f75-b959-05efdcb00451', metadata={'source': './txt/한국과는 달라도 너무 다른, 미국 LA [문화 전반].txt'}, page_content='나는 2년 전, 한국에서 미국 LA 땅을 처음 밟았다. 나는 그 이후 LA의 한인 타운에서 계속 살았다. 한인타운은 한국의 문화를 많이 가지고 있어서, 다른 지역에서 살고 있는 한인에 비해 문화차이로 인한 어려움을 많이 겪지 않은 편이다. 그럼에도 불구하고 분명히 한국과는 다른 미국만의 문화가 있다. 미국의 독특한 문화는 아닐지라도, 한국에서는 경험하지 못한 색다른 것들이 때로는 나를 당황스럽게도, 때로는 놀랍게도 만들었다. 아마 미국에 오지 않았다면 경험하지 못했을 몇 가지의 이색적인 미국의 문화들을 여러분들과 나누고 싶다. 1. 재채기할 때 건네는 인사 Bless you 처음 이 단어를 들었던 것은 내가 택시를 타고 있을 때였다. 나는 택시를 타고 집에 가던 도중 재채기를 했다. 에취! 그 순간 택시 기사가 Bless you라고 말했다. 처음 나는 이 말의 의미를 알지 못해서 아무런 대답도 하지 못한 채 어리둥절했다. (내가 이 단어를 제대로 들은 것인지도 확신이 없었다.) 그 이후 미국에서 자란 친구가 이야기해 주길, 재채기를 하면 Bless you라고 말해주는 것이 미국에서는 일종의 예의라고 했다. 그 친구는 덧붙여 자신이 처음으로 한국사람들과 친해졌을 무렵, 한국사람들은 타인이 재채기를 해도 아무런 리액션을 해주지 않아 오히려 자신이 의아하게 느꼈다고 한다. 그 친구는 오히려 아무런 반응을 하지 않는 한국의 문화를 어색하게 느꼈던 것이다. 나는 왜 미국인들은 재채기를 할 때 인사를 건네는 것인지 궁금해졌다. 그래서 찾아보니, 이는 영국에서 페스트에 대한 공포가 휩쓸던 시대에 생겨난 말이라고 한다. 당시엔 재채기가 위험한 것으로 여겨졌다. 큰 질병에 걸릴 조짐이라든지 영혼이 

In [14]:
# 개선된 시스템 프롬프트 - 문화/예절 정보 포함
SYSTEM_PROMPT = """
당신은 여행 전문가로, 각국의 여행 정보를 포괄적으로 제공하는 챗봇입니다.
다음 두 가지 주요 정보를 정확하고 유용하게 제공해야 합니다:

1. 각국의 입국 시 금지/제한되는 품목에 대한 정보
2. 각국의 문화, 예절, 관습에 대한 정보

중요 지침:
1. 사용자 질문을 정확히 분석하고 관련 정보를 제공하세요.
2. 답변은 5-7문장으로 제공하되, 자연스러운 흐름을 유지하세요.
3. 질문과 관련 없는 정보는 포함하지 마세요.
4. 문서에서 찾을 수 없는 정보에 대해서는 gemma 모델의 일반 지식을 활용하되, "문서에서 직접적인 정보를 찾지 못했으나 일반적으로..."와 같은 구분을 명확히 하세요.
5. 사용자가 여행이나 방문에 대해 언급했다면, 해당 국가의 금지/제한 품목 정보와 함께 현지 문화와 예절 정보를 포함하세요.

항공기 탑승 시 일반적인 제한사항:
- 액체류: 개별 용기 100ml 이하, 1L 투명 지퍼백에 담아야 함
- 예외: 유아식, 의약품 등(보안 검색 시 별도 신고)
- 위험물: 라이터(1개만 기내 반입 가능), 성냥, 스프레이, 압축 가스
- 날카로운 물건: 칼, 가위, 면도칼 등은 위탁 수하물로만 가능
"""

# 개선된 프롬프트 템플릿
PROMPT_TEMPLATE = """
{system_prompt}

사용자 질문: {question}

다음은 질문과 관련된 문서 정보입니다:
{context}

지침:
1. 위 정보를 바탕으로 사용자 질문에 답변하세요.
2. 문서에서 충분한 정보를 찾지 못했다면, 일반적인 여행 지식을 활용하여 답변을 보완하세요.
3. 특히 다음 내용을 포함하도록 노력하세요:
   - 금지/제한 품목 정보
   - 현지 문화와 예절 정보
   - 여행자에게 유용한 팁
4. 답변은 5-7문장 정도로 자연스럽게 작성하세요.
5. 불확실한 정보가 있다면 그 불확실성을 명시하세요.

답변:
"""

# 개선된 국가와 품목, 문화/예절 감지 클래스
class EnhancedEntityDetector:
    def __init__(self):
        # 국가 목록
        self.countries = {
            "일본": ["일본", "japan", "japanese", "jp", "니혼", "닛폰"],
            "미국": ["미국", "usa", "united states", "america", "american", "us", "유에스에이", "아메리카"],
            "중국": ["중국", "china", "chinese", "cn", "차이나"],
            "태국": ["태국", "thailand", "thai", "th"],
            "호주": ["호주", "australia", "australian", "au", "오스트레일리아"],
            "한국": ["한국", "korea", "korean", "kr", "대한민국", "코리아"],
            "영국": ["영국", "united kingdom", "uk", "britain", "british", "gb", "잉글랜드", "브리튼"],
            "프랑스": ["프랑스", "france", "french", "fr"],
            "독일": ["독일", "germany", "german", "de"],
            "이탈리아": ["이탈리아", "italy", "italian", "it"],
            "스페인": ["스페인", "spain", "spanish", "es"],
            "캐나다": ["캐나다", "canada", "canadian", "ca"],
            "싱가포르": ["싱가포르", "singapore", "singaporean", "sg"],
            "말레이시아": ["말레이시아", "malaysia", "malaysian", "my"],
            "인도네시아": ["인도네시아", "indonesia", "indonesian", "id"],
            "베트남": ["베트남", "vietnam", "vietnamese", "vn"],
            "필리핀": ["필리핀", "philippines", "philippine", "filipino", "ph"],
            "대만": ["대만", "taiwan", "taiwanese", "tw", "타이완"]
        }
        
        # 품목 목록
        self.items = {
            "액체류": ["액체", "물", "음료", "주스", "음료수", "화장품", "샴푸", "로션", "향수"],
            "식품류": ["음식", "식품", "과일", "채소", "육류", "생선", "해산물", "유제품", "간식", "과자"],
            "의약품": ["약", "의약품", "처방약", "알약", "진통제", "비타민", "영양제", "보조제"],
            "주류": ["술", "주류", "알코올", "맥주", "와인", "소주", "위스키", "보드카"],
            "담배류": ["담배", "시가", "전자담배", "베이프", "흡연", "니코틴"],
            "전자제품": ["전자제품", "노트북", "컴퓨터", "휴대폰", "카메라", "배터리", "충전기", "전자기기"],
            "무기류": ["칼", "가위", "면도기", "날카로운", "무기", "총", "총기", "폭발물"],
            "식물류": ["식물", "씨앗", "화초", "꽃", "나무", "농산물"],
            "동물류": ["동물", "애완동물", "반려동물", "강아지", "고양이", "새", "어류"],
            "위조품": ["가짜", "위조", "복제품", "짝퉁", "모조품"],
            "수하물": ["수하물", "짐", "캐리어", "가방", "여행가방"]
        }
        
        # 추가: 문화/예절 키워드
        self.culture_keywords = [
            "문화", "예절", "관습", "매너", "예의", "인사", "습관", "금기", "식사", "식사예절", 
            "식사 예절", "문화차이", "문화 차이", "에티켓", "금지", "주의", "행동", "관행", 
            "전통", "규범", "의례", "행동양식", "행동 양식", "사회규범", "사회 규범"
        ]
    
    def detect_country(self, text: str) -> List[str]:
        """텍스트에서 국가명 감지"""
        detected = []
        
        text_lower = text.lower()
        
        for country, aliases in self.countries.items():
            for alias in aliases:
                if re.search(r'\b' + re.escape(alias.lower()) + r'\b', text_lower):
                    detected.append(country)
                    break
        
        return detected
    
    def detect_item(self, text: str) -> List[str]:
        """텍스트에서 품목명 감지"""
        detected = []
        
        text_lower = text.lower()
        
        for item, aliases in self.items.items():
            for alias in aliases:
                if re.search(r'\b' + re.escape(alias.lower()) + r'\b', text_lower):
                    detected.append(item)
                    break
        
        return detected
    
    def detect_culture_interest(self, text: str) -> bool:
        """문화/예절에 대한 관심 감지"""
        text_lower = text.lower()
        
        for keyword in self.culture_keywords:
            if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text_lower):
                return True
        
        return False
    
    def detect_travel_intent(self, text: str) -> bool:
        """여행 의도가 있는지 감지"""
        travel_keywords = ["여행", "방문", "관광", "출장", "입국", "출국", "갈 예정", "여행 계획", "가려고", "갈 거"]
        text_lower = text.lower()
        
        for keyword in travel_keywords:
            if keyword in text_lower:
                return True
        
        return False

# 개선된 쿼리 확장 및 필터링 클래스
class EnhancedQueryProcessor:
    def __init__(self, entity_detector: EnhancedEntityDetector):
        self.entity_detector = entity_detector
    
    def enhance_query(self, query: str) -> str:
        """
        사용자 질문을 확장하여 더 관련성 높은 문서를 검색하도록 함
        문화/예절 정보도 포함하도록 개선
        """
        countries = self.entity_detector.detect_country(query)
        items = self.entity_detector.detect_item(query)
        is_travel_intent = self.entity_detector.detect_travel_intent(query)
        is_culture_interest = self.entity_detector.detect_culture_interest(query)
        
        # 기본 쿼리
        enhanced_query = query
        
        # 여행 의도가 감지되었지만 여행 관련 키워드가 없는 경우에만 추가
        if is_travel_intent and not any(keyword in query.lower() for keyword in ["항공", "비행기", "여행", "입국", "세관", "반입"]):
            enhanced_query += " 여행"
        
        # 국가가 감지되면 국가 관련 키워드 추가
        if countries:
            country_terms = " ".join(countries)
            enhanced_query += f" {country_terms}"
            
            # 품목이 감지되면 품목 관련 키워드 추가
            if items:
                item_terms = " ".join(items)
                # 금지/제한 관련 키워드 추가
                if not any(keyword in query.lower() for keyword in ["금지", "제한", "반입금지", "통관", "규제"]):
                    enhanced_query += f" {item_terms} 반입 제한 규제"
                else:
                    enhanced_query += f" {item_terms}"
            
            # 문화/예절 관심이 감지되었거나, 명시적으로 물품 관련 질문이 아닌 경우
            if is_culture_interest or not items:
                enhanced_query += " 문화 예절 관습 유의사항"
        
        return enhanced_query

# 개선된 응답 포맷팅 클래스
class EnhancedResponseFormatter:
    def format_response(self, response: str) -> str:
        """
        응답을 더 나은 형식으로 정리
        """
        # 중복된 공백 제거
        formatted = re.sub(r'\s+', ' ', response).strip()
        
        # 불필요한 마크다운 또는 특수 문자 제거
        formatted = re.sub(r'#+\s+', '', formatted)  # 마크다운 헤더 제거
        formatted = re.sub(r'\*+', '', formatted)    # 마크다운 볼드/이탤릭 제거
        
        # 출처 또는 참고문헌 관련 문구 제거
        formatted = re.sub(r'출처:.*', '', formatted)
        formatted = re.sub(r'참고:.*', '', formatted)
        formatted = re.sub(r'참고문헌:.*', '', formatted)
        
        # 링크 형식 제거
        formatted = re.sub(r'\[.*?\]\(.*?\)', '', formatted)
        
        return formatted.strip()

# 개선된 여행 정보 챗봇 클래스
class EnhancedTravelInfoBot:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 5})  # 더 많은 문서 검색
        self.entity_detector = EnhancedEntityDetector()
        self.query_processor = EnhancedQueryProcessor(self.entity_detector)
        self.response_formatter = EnhancedResponseFormatter()
        
        # LangChain 프롬프트 템플릿 설정
        self.prompt = PromptTemplate(
            input_variables=["system_prompt", "question", "context"],
            template=PROMPT_TEMPLATE
        )
    
    def get_response(self, query: str, use_rag: bool = True) -> str:
        """
        사용자 질문에 대한 응답 생성 (더 유연한 버전)
        
        Args:
            query: 사용자 질문
            use_rag: RAG를 사용할지 여부 (False면 gemma LLM만 사용)
            
        Returns:
            생성된 응답
        """
        # 빈 질문 처리
        if not query.strip():
            return "질문을 입력해주세요."
        
        # 질문이 너무 짧은 경우
        if len(query.strip()) < 2:
            return "좀 더 자세한 질문을 입력해주세요."
            
        try:
            # 국가 및 문화/예절 관심 감지
            countries = self.entity_detector.detect_country(query)
            is_culture_interest = self.entity_detector.detect_culture_interest(query)
            
            # 컨텍스트 초기화
            context = ""
            
            # RAG 사용 설정된 경우에만 문서 검색
            if use_rag:
                # 질문 강화
                enhanced_query = self.query_processor.enhance_query(query)
                
                # 관련 문서 검색
                docs = self.retriever.get_relevant_documents(enhanced_query)
                
                # 검색된 문서가 있는 경우 컨텍스트 준비
                if docs:
                    doc_contents = [doc.page_content for doc in docs]
                    context = "\n\n---\n\n".join(doc_contents)
            
            # Ollama API 호출을 위한 메시지 준비
            messages = [
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT
                }
            ]
            
            # 컨텍스트가 있는 경우 (RAG 모드)
            if context:
                messages.append({
                    "role": "user",
                    "content": f"질문: {query}\n\n다음은 참고할 정보입니다:\n{context}\n\n자연스러운 5-7문장으로 답변해주세요."
                })
            # 컨텍스트가 없는 경우 (LLM 단독 모드)
            else:
                # 국가가 감지된 경우 더 구체적인 지침 제공
                if countries:
                    country_str = ", ".join(countries)
                    
                    # 문화/예절 관심이 감지된 경우
                    if is_culture_interest:
                        messages.append({
                            "role": "user",
                            "content": f"질문: {query}\n\n{country_str}의 문화와 예절에 대해 자연스러운 5-7문장으로 답변해주세요."
                        })
                    # 일반 여행 질문인 경우
                    else:
                        messages.append({
                            "role": "user",
                            "content": f"질문: {query}\n\n{country_str}의 여행 정보(금지/제한 품목 및 문화/예절)에 대해 자연스러운 5-7문장으로 답변해주세요."
                        })
                # 국가가 감지되지 않은 경우 일반적인 질문으로 처리
                else:
                    messages.append({
                        "role": "user",
                        "content": f"질문: {query}\n\n위 질문에 대해 자연스러운 5-7문장으로 답변해주세요."
                    })
            
            # Ollama API 호출 - 최적의 모델 선택 (7b 우선 사용)
            try:
                model_to_use = "gemma3:7b" if any("gemma3:7b" in model['name'] for model in ollama.list()['models']) else "gemma3:1b"
            except:
                model_to_use = "gemma3:1b"  # 기본 폴백 모델
                
            response = ollama.chat(
                model=model_to_use,
                messages=messages
            )
            
            raw_response = response["message"]["content"]
            
            # 응답 포맷팅
            formatted_response = self.response_formatter.format_response(raw_response)
            
            # 응답이 너무 짧은 경우
            if len(formatted_response) < 10:
                return "죄송합니다. 해당 질문에 대한 충분한 정보를 찾을 수 없습니다. 더 구체적인 질문을 해주시거나 공식 여행 정보를 확인해보세요."
                
            return formatted_response
            
        except Exception as e:
            # 오류 발생 시 안전한 응답 제공
            print(f"오류 발생: {str(e)}")
            return "죄송합니다. 응답 생성 중 오류가 발생했습니다. 다시 시도해주세요."

# 챗봇 인스턴스 생성 및 사용
def setup_chatbot(vector_store_path="faiss_index"):
    """
    벡터 저장소에서 챗봇 인스턴스 생성
    
    Args:
        vector_store_path: 벡터 저장소 경로
        
    Returns:
        챗봇 인스턴스
    """
    # 임베딩 모델 로드
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    # 벡터 저장소 로드 - allow_dangerous_deserialization 플래그 추가
    vector_store = FAISS.load_local(
        vector_store_path, 
        embeddings, 
        allow_dangerous_deserialization=True
    )
    
    # 챗봇 인스턴스 생성
    return EnhancedTravelInfoBot(vector_store)

# 개선된 대화형 모드
def interactive_chat_mode(bot):
    """
    대화형 모드로 챗봇 실행 (RAG와 LLM 전환 기능 추가)
    
    Args:
        bot: 챗봇 인스턴스
    """
    print("=== 개선된 여행 정보 챗봇 ===")
    print("종료하려면 'q' 또는 'exit' 또는 '종료'를 입력하세요.")
    print("RAG 모드 끄기: 'rag off', RAG 모드 켜기: 'rag on'")
    print("="*50)
    print("\n안녕하세요! 여행 정보에 대해 궁금한 점을 질문해주세요.")
    print("예시 질문:")
    print("- '일본에 과일을 가져갈 수 있나요?'")
    print("- '미국 입국 시 가져가면 안되는 물건은?'")
    print("- '태국의 문화적 금기사항은?'")
    print("- '프랑스에서 알아야 할 식사 예절은?'\n")
    
    # RAG 모드 (기본값: True)
    use_rag = True
    
    while True:
        try:
            user_input = input("\n질문을 입력하세요: ")
            print(f'질문: {user_input}\n')
            user_input = user_input.strip()
            
            # 종료 명령 확인
            if user_input.lower() in ['q', 'exit', '종료', 'quit']:
                print("챗봇을 종료합니다. 좋은 여행 되세요!")
                break
            
            # RAG 모드 전환 명령 확인
            if user_input.lower() == 'rag off':
                use_rag = False
                print("\n[INFO] RAG 모드가 꺼졌습니다. 이제 gemma 모델만 사용합니다.")
                continue
            elif user_input.lower() == 'rag on':
                use_rag = True
                print("\n[INFO] RAG 모드가 켜졌습니다. 문서 검색 결과를 활용합니다.")
                continue
                
            rag_status = "RAG 모드" if use_rag else "LLM 단독 모드"
            print(f"\n[{rag_status}로 응답 생성 중...]")
            
            response = bot.get_response(user_input, use_rag=use_rag)
            print(f"답변: {response}")
            
        except KeyboardInterrupt:
            print("\n\n사용자에 의해 챗봇이 종료되었습니다.")
            break
        except Exception as e:
            print(f"\n오류가 발생했습니다: {str(e)}")
            print("다시 시도해주세요.")

# 메인 실행 함수
def main():
    # 벡터 저장소가 이미 생성되어 있다면 바로 로드
    if os.path.exists("faiss_index"):
        bot = setup_chatbot()
        interactive_chat_mode(bot)
    else:
        print("벡터 저장소를 찾을 수 없습니다. 먼저 데이터를 로드하고 벡터 저장소를 생성해주세요.")

if __name__ == "__main__":
    main()

=== 개선된 여행 정보 챗봇 ===
종료하려면 'q' 또는 'exit' 또는 '종료'를 입력하세요.
RAG 모드 끄기: 'rag off', RAG 모드 켜기: 'rag on'

안녕하세요! 여행 정보에 대해 궁금한 점을 질문해주세요.
예시 질문:
- '일본에 과일을 가져갈 수 있나요?'
- '미국 입국 시 가져가면 안되는 물건은?'
- '태국의 문화적 금기사항은?'
- '프랑스에서 알아야 할 식사 예절은?'

질문: 일본에서는 밥을 먹을 때 어떻게 먹어?


[RAG 모드로 응답 생성 중...]

답변: 일본 문화는 종교, 음식, 예술, 생활문화, 직장문화 등 다양한 요소로 구성되어 있으며, 특히 식사 방식에서 소리 내어 먹는 문화가 유래되었습니다. 일본은 남북으로 긴 지형으로 식재료가 다르고, 계절별로 맛있는 음식을 즐길 수 있습니다. 편의점 음식은 퀄리티가 높아 맛집보다 더 낫다는 평이 있으며, 캡슐 호텔은 일본 여행을 위한 필수 앱입니다. 일본 문화는 외국인들에게도 매력적인 요소이며, 일본 여행을 통해 다양한 문화를 경험할 수 있습니다.
질문: 한국은 밥을 잘 먹어?


[RAG 모드로 응답 생성 중...]



사용자에 의해 챗봇이 종료되었습니다.
